<a href="https://colab.research.google.com/github/stfromnjust/nlp-task/blob/main/Seq2SeqForTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/stfromnjust/nlp-task/raw/main/corpus.zip
!unzip -d ./corpus corpus.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./corpus/corpus/t5499.txt  
  inflating: ./corpus/corpus/t55.txt  
  inflating: ./corpus/corpus/t550.txt  
  inflating: ./corpus/corpus/t5500.txt  
  inflating: ./corpus/corpus/t5501.txt  
  inflating: ./corpus/corpus/t5502.txt  
  inflating: ./corpus/corpus/t5503.txt  
  inflating: ./corpus/corpus/t5504.txt  
  inflating: ./corpus/corpus/t5505.txt  
  inflating: ./corpus/corpus/t5506.txt  
  inflating: ./corpus/corpus/t5507.txt  
  inflating: ./corpus/corpus/t5508.txt  
  inflating: ./corpus/corpus/t5509.txt  
  inflating: ./corpus/corpus/t551.txt  
  inflating: ./corpus/corpus/t5510.txt  
  inflating: ./corpus/corpus/t5511.txt  
  inflating: ./corpus/corpus/t5512.txt  
  inflating: ./corpus/corpus/t5513.txt  
  inflating: ./corpus/corpus/t5514.txt  
  inflating: ./corpus/corpus/t5515.txt  
  inflating: ./corpus/corpus/t5516.txt  
  inflating: ./corpus/corpus/t5517.txt  
  inflating: ./corpus/corpus/t5518.txt  
  inflating: ./corpus/corpus/t5519

In [2]:
seed = 2021

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.legacy.data import *

import time
import math
import random
import jieba
import re

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 清理, 读取数据

In [5]:
def clean_data(sentence):
  re_list = [
            r'(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?',
            r'!\[.*?\]\(.*?\)',
            r'\[.*?\]\(.*?\)',
            r'多知网([1-9]|1[0-2])月([0-9]*)日消息，',
            r'（多知网.*）',
            r'&version=.*'
  ]
  for sub_re in re_list:
    sentence = re.sub(sub_re, '', sentence)
  return sentence

In [6]:
def extract_data(data):
  data = data.split('\n')
  summary = data[0]
  text = data[1]
  reg = re.compile(r'^summary{{(?P<S>(.)*)}}$')
  reg_match = reg.match(summary)
  if reg_match:
    line_bits = reg_match.groupdict()
    summary = line_bits['S']
  reg = re.compile(r'^text{{(?P<S>(.)*)}}$')
  reg_match = reg.match(text)
  if reg_match:
    line_bits = reg_match.groupdict()
    text = line_bits['S']
  return summary, text

In [7]:
# test
with open('./corpus/corpus/t12452.txt', 'r', encoding='utf-8') as f:
    data = f.read()
    summary, text = extract_data(data)
    summary = clean_data(summary)
    text = clean_data(text)
    print(summary)
    print(text)

从推卸责任的角度采取对策，是转移视线，反而忽略了真相和真问题
*本文根据北京师范大学教育学部张燕教授于2017年12月13日在21世纪教育研究院主办的“学前教育的供给侧改革”专题研讨会上的发言整理。李相禹老师对此文内容亦有贡献。### 从推卸责任的角度采取对策将导致真问题被忽略近日虐童事件引发了广泛关注，媒体连篇累牍，社会影响巨大。家长对幼儿教育产生了很大质疑。如何看待这一事件？我们认为，这类极端恶性事件，在庞大的中国幼儿园基数中，是个别的特殊性问题。相关部门对事件反应过度，把个别推及普遍，加码监管，甚至惩罚整个基层幼教，对从业人员及队伍造成伤害不可取。否则，会致使机构和教师自保，最终令儿童受害；以证照为名大量排查封杀无资格幼儿园，也会导致供给极度不足而无法应对需求；督导全覆盖加剧了老师与家长之间的不信任和对立关系等——凡此等等，无益于问题的解决，反而制造了新问题，出现恶性循环。从推卸责任的角度采取对策，是转移视线反而忽略了真相和真问题！近年幼教恶性事件频出，一方面与现代化传媒手段进步及媒体放大效应有关；另一方面，确实到了要问一个“为什么？”的时候，需要认真思考我国幼儿教育不断发生状况究竟存在怎样的问题？有必要对幼儿教育十余年来即2010年（国家中长期规划）前后的发展现状做一下梳理，澄清问题所在及其由来。### 十余年来幼儿教育的发展现状2010年，国家中长期教育规划颁发以来，幼儿教育得到政府空前关注和重视，国人欢呼“幼儿教育春天来了！”的确，短短数年来，学前教育相继出台了三个“三年规划”，规划的实施带来了事业的跨越式发展，幼教规模、数据一路攀升： **1 2010年以来幼儿园数、班级数，幼儿在园数与入园率急剧攀升**  * 自2009年至2014年，全国幼儿园总量增幅达到51.88%；班级数量增长了59.26%。专任教师数量增长了87.05%，保育员数量增长了59.75%。##### 来源：教育部. 教育规划纲要实施5周年学前教育专题评估##### 来源：教育部. 教育规划纲要实施5周年学前教育专题评估  * 2014年，全国在园幼儿达到4050.71万人，比2009年增加52.41%。 2016年，在园儿童4413.86万人。##### 来源：教育部. 教育规划纲要实施5周年学前教育专题评估  * 学前教育三年毛入园率2009年仅为50.9%，2014年达到70

In [8]:
file_path = './corpus/corpus/'
summary_data = []
text_data = []
data_num = 24476
for i in range(data_num):
  file_name = 't' + str(i + 1) + '.txt'
  with open(file_path + file_name, 'r', encoding='utf-8') as f:
    data = f.read()
    summary, text = extract_data(data)
    summary = clean_data(summary)
    text = clean_data(text)
    summary_data.append(summary)
    text_data.append(text)
# 前10个
print(len(summary_data), len(text_data))
print('summarys:', summary_data[:10])
print('texts', text_data[:10])

24476 24476
summarys: ['方直科技拟以自有资金1.2亿元共同投资设立嘉道方直投资基金', '2016年9月1日学生总人数31788人，较去年同期的27644人，增加了15%。', '2016年，胡歌正式代言英孚教育。', '今日，决胜网宣布引入前21世纪不动产集团副总裁兼首席信息官王雷和前如新中国销售总监张鹏，分别担任决胜网新业务及企业发展副总裁和人力资源副总裁职务。', '传统线下机构在发展线上时，也因为盗版问题有所顾虑。随着法律意思的增强，越来越多的在线教育公司已经着手维护课程版权，保护应有的权益。', '据说，这些文字价值十亿美金。', '乐视教育也开始切入VR领域。目前，乐视教育与新东方已经达成初步合作意向，双方将在英语课堂应用VR教学，希望为学员的在线课堂带来场景式体验。', '家长和孩子的真正痛点是什么，家长到底会为什么付费？为什么说家教O2O是刚需？为什么说轻轻家教不是Uber模式，也不是名师模式？为什么停掉补贴？', '有多名网友举报英语（二）出现泄题，有学生称考前45分钟收到考研机构的答案。南都记者对比发现，该考前答案与试题内容完全吻合，但答案正确率约五六成。', '多知网致电文都考研官方，对方表示：“文都非常重视记者所反映的问题和所提供的线索， 正积极展开内部核查。文都教育多年从事考研培训业务，一直合法合规经营。”文都方面表示，内部调查结果将及时向媒体通报。']
texts ['今日，方直科技发公告，拟用自有资金人民币1.2亿元，与深圳嘉道谷投资管理有限公司、深圳嘉道功程股权投资基金（有限合伙）共同发起设立嘉道方直教育产业投资基金（暂定名）。该基金认缴出资总规模为人民币3.01亿元。基金的出资方式具体如下：出资进度方面，基金合伙人的出资应于基金成立之日起四年内分四期缴足，每期缴付7525万元；各基金合伙人每期按其出资比例缴付。合伙期限为11年，投资目标为教育领域初创期或成长期企业。截止公告披露日，深圳嘉道谷投资管理有限公司股权结构如下:截止公告披露日，深圳嘉道功程股权投资基金产权结构如下:公告还披露，方直科技将探索在中小学教育、在线教育、非学历教育、学前教育、留学咨询等教育行业其他分支领域的投资。方直科技2016年营业收入9691万元，营业利润1432万元，归属于普通股股东的净利润1847万元。', '今日，睿见教育发布中期

# jieba分词

In [10]:
# # 按字符级切割，并添加<eos>
# en_token_list = [[char for char in line]+["<eos>"] for line in en_data]
# ch_token_list = [[char for char in line]+["<eos>"] for line in ch_data]
# print('英文数据:\n', en_token_list[:2])
# print('\n中文数据:\n', ch_token_list[:2])
# jieba分词, 并添加<eos>
summary_token_list = [list(jieba.cut(summary))+['<eos>'] for summary in summary_data]
text_token_list  = [list(jieba.cut(text))+['<eos>'] for text in text_data]
print(summary_token_list[:2])
print(text_token_list[:2])

[['方直', '科技', '拟以', '自有', '资金', '1.2', '亿元', '共同', '投资', '设立', '嘉道', '方直', '投资', '基金', '<eos>'], ['2016', '年', '9', '月', '1', '日', '学生', '总', '人数', '31788', '人', '，', '较', '去年同期', '的', '27644', '人', '，', '增加', '了', '15%', '。', '<eos>']]
[['今日', '，', '方直', '科技', '发', '公告', '，', '拟用', '自有', '资金', '人民币', '1.2', '亿元', '，', '与', '深圳', '嘉道', '谷', '投资', '管理', '有限公司', '、', '深圳', '嘉道', '功程', '股权', '投资', '基金', '（', '有限', '合伙', '）', '共同', '发起', '设立', '嘉道', '方直', '教育', '产业', '投资', '基金', '（', '暂定名', '）', '。', '该', '基金', '认缴', '出资', '总', '规模', '为', '人民币', '3.01', '亿元', '。', '基金', '的', '出资', '方式', '具体', '如下', '：', '出资', '进度', '方面', '，', '基金', '合伙人', '的', '出资', '应', '于', '基金', '成立', '之日起', '四年', '内', '分', '四期', '缴足', '，', '每期', '缴付', '7525', '万元', '；', '各', '基金', '合伙人', '每期', '按', '其', '出资', '比例', '缴付', '。', '合伙', '期限', '为', '11', '年', '，', '投资', '目标', '为', '教育领域', '初创期', '或', '成长期', '企业', '。', '截止', '公告', '披露', '日', '，', '深圳', '嘉道', '谷', '投资', '管理', '有限公司', '股权结构', '如下', ':', '截止', '公告', '披露', '日', '

# 词汇表, 字典映射

In [12]:
TEXT = Field()
TEXT.build_vocab(summary_token_list + text_token_list, specials=['<pad>', '<unk>', '<bos>', '<eos>'])
print(type(TEXT.vocab))
print(len(TEXT.vocab))
print(TEXT.vocab.itos[:20])
print(TEXT.vocab.stoi['<unk>'])

<class 'torchtext.vocab.Vocab'>
245404
['<unk>', '<pad>', '<bos>', '<eos>', '，', '的', '。', '*', ' ', '、', '在', '是', '教育', '和', '了', '“', '”', '有', '也', '：']
0


In [13]:
summary_num_list = [[TEXT.vocab.stoi[token] for token in line]for line in summary_token_list]
text_num_list = [[TEXT.vocab.stoi[token] for token in line]for line in text_token_list]
print(summary_num_list[0])
print(text_num_list[0]) 

[4398, 159, 2414, 1933, 445, 3393, 153, 573, 93, 713, 22658, 4398, 93, 409, 3]
[776, 4, 4398, 159, 1860, 658, 4, 27415, 1933, 445, 395, 3393, 153, 4, 27, 1317, 22658, 9136, 93, 170, 342, 9, 1317, 22658, 57193, 423, 93, 409, 38, 1268, 2287, 37, 573, 1937, 713, 22658, 4398, 12, 380, 93, 409, 38, 11345, 37, 6, 264, 409, 5934, 1391, 756, 374, 22, 395, 26593, 153, 6, 409, 5, 1391, 134, 666, 1457, 19, 1391, 2077, 150, 4, 409, 1086, 5, 1391, 1259, 104, 409, 281, 7948, 2786, 250, 656, 22277, 141526, 4, 9025, 42105, 158638, 126, 28, 652, 409, 1086, 9025, 1044, 144, 1391, 601, 42105, 6, 2287, 4585, 22, 501, 20, 4, 93, 356, 22, 580, 17085, 196, 8014, 105, 6, 1522, 658, 969, 142, 4, 1317, 22658, 9136, 93, 170, 342, 4703, 1457, 896, 1522, 658, 969, 142, 4, 1317, 22658, 57193, 423, 93, 409, 6576, 1038, 1457, 896, 658, 85, 969, 4, 4398, 159, 34, 563, 10, 561, 12, 9, 130, 9, 4526, 12, 9, 973, 9, 195, 653, 25, 12, 76, 254, 7180, 135, 5, 93, 6, 4398, 159, 143, 20, 569, 177, 160478, 126, 4, 569, 723, 930

# 表示为Dataset

In [14]:
class SummarizationDataset(Dataset):
    def __init__(self, src_data, trg_data):
        self.src_data = src_data
        self.trg_data = trg_data

        assert len(src_data) == len(trg_data), \
            "numbers of src_data  and trg_data must be equal!"

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src_sample =self.src_data[idx]
        src_len = len(self.src_data[idx])
        trg_sample = self.trg_data[idx]
        trg_len = len(self.trg_data[idx])
        return {"src": src_sample, "src_len": src_len, "trg": trg_sample, "trg_len": trg_len}

In [15]:
def padding_batch(batch):
    """
    input: -> list of dict
        [{'src': [1, 2, 3], 'trg': [1, 2, 3]}, {'src': [1, 2, 2, 3], 'trg': [1, 2, 2, 3]}]
    output: -> dict of tensor 
        {
            "src": [[1, 2, 3, 0], [1, 2, 2, 3]].T
            "trg": [[1, 2, 3, 0], [1, 2, 2, 3]].T
        }
    """
    src_lens = [d["src_len"] for d in batch]
    trg_lens = [d["trg_len"] for d in batch]
    
    src_max = max([d["src_len"] for d in batch])
    trg_max = max([d["trg_len"] for d in batch])
    for d in batch:
        d["src"].extend([TEXT.vocab.stoi['<pad>']] * (src_max - d["src_len"]))
        d["trg"].extend([TEXT.vocab.stoi['<pad>']] * (trg_max - d["trg_len"]))
    srcs = torch.tensor([pair["src"] for pair in batch], dtype=torch.long, device=device)
    trgs = torch.tensor([pair["trg"] for pair in batch], dtype=torch.long, device=device)
    
    batch = {"src":srcs.T, "src_len":src_lens, "trg":trgs.T, "trg_len":trg_lens}
    return batch

# Attention机制

## 模型

In [16]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
    def forward(self, input_seqs, input_lengths, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [seq_len, batch, embed_dim]
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        
        outputs, hidden = self.gru(packed, hidden)        
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
        # output_lengths = [batch]
        return outputs, hidden

In [17]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)  # [seq_len, batch]

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)  # [seq_len, batch, hid_dim]
        return torch.sum(hidden * energy, dim=2)  # [seq_len, batch]

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1) -> [seq_len, batch, N]
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        # energy = [sql_len, batch, hidden_size]
        return torch.sum(self.v * energy, dim=2)  # [seq_len, batch]

    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch,  n_directions * hid_dim]
        # encoder_outputs = [seq_len, batch, hid dim * n directions]
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        attn_energies = attn_energies.t()  # [batch, seq_len]
 
        return F.softmax(attn_energies, dim=1).unsqueeze(1)  # softmax归一化# [batch, 1, seq_len]

In [18]:
class AttnDecoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers=1, dropout=0.5, bidirectional=True, attn_method="general"):
        super(AttnDecoder, self).__init__()

        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.concat = nn.Linear(hid_dim * 2 * 2, hid_dim*2)
            self.out = nn.Linear(hid_dim*2, output_dim)
            self.attn = Attn(attn_method, hid_dim*2)
        else:
            self.concat = nn.Linear(hid_dim * 2, hid_dim)
            self.out = nn.Linear(hid_dim, output_dim)
            self.attn = Attn(attn_method, hid_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, token_inputs, last_hidden, encoder_outputs):
        batch_size = token_inputs.size(0)
        embedded = self.embedding(token_inputs)
        embedded = self.embedding_dropout(embedded)
        embedded = embedded.view(1, batch_size, -1) # [1, B, hid_dim]

        gru_output, hidden = self.gru(embedded, last_hidden)
        # gru_output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]

        # encoder_outputs = [sql_len, batch, hid dim * n directions]
        attn_weights = self.attn(gru_output, encoder_outputs)
        # attn_weights = [batch, 1, sql_len]
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # [batch, 1, hid_dim * n directions]

        # LuongAttention
        gru_output = gru_output.squeeze(0) # [batch, n_directions * hid_dim]
        context = context.squeeze(1)       # [batch, n_directions * hid_dim]
        concat_input = torch.cat((gru_output, context), 1)  # [batch, n_directions * hid_dim * 2]
        concat_output = torch.tanh(self.concat(concat_input))  # [batch, n_directions*hid_dim]

        output = self.out(concat_output)  # [batch, output_dim]
        output = self.softmax(output)

        return output, hidden, attn_weights

In [27]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 token2idx=None,   # token2index
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict  # 训练阶段还是预测阶段
        self.token2idx = token2idx  # decoder的字典，存放特殊token对应的id
        self.max_len = max_len  # 翻译时最大输出长度

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert encoder.gru.bidirectional == decoder.gru.bidirectional, \
            "Decoder and encoder must had same value of bidirectional attribute!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, teacher_forcing_ratio=0.5):
        # input_batches = [seq_len, batch]
        # target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        
        BOS_token = self.token2idx["<bos>"]
        EOS_token = self.token2idx["<eos>"]
        PAD_token = self.token2idx["<pad>"]

        # 初始化
        enc_n_layers = self.encoder.gru.num_layers
        enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        encoder_hidden = torch.zeros(enc_n_layers*enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_outputs = [input_lengths, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim]
        encoder_outputs, encoder_hidden = self.encoder(
            input_batches, input_lengths, encoder_hidden)

        # 初始化
        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        decoder_hidden = encoder_hidden

        if self.predict:
            # 一次只输入一句话
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

            while True:
                decoder_output, decoder_hidden, decoder_attn = self.decoder(
                    decoder_input, decoder_hidden, encoder_outputs
                )
                # [1, 1]
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(1).detach()
                output_token = topi.squeeze().detach().item()
                if output_token == EOS_token or len(output_tokens) == self.max_len:
                    break
                output_tokens.append(output_token)
            return output_tokens

        else:
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # decoder_output = [batch, output_dim]
                    # decoder_hidden = [n_layers*n_directions, batch, hid_dim]
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t]  # 下一个输入来自训练数据
                else:
                    decoder_output, decoder_hidden, decoder_attn = self.decoder(
                        decoder_input, decoder_hidden, encoder_outputs
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1).detach()  # 下一个输入来自模型预测
            
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1, self.decoder.output_dim),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)               # [batch*seq_len]
            )
            return loss

# 训练和预测代码

In [21]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
def train(
    model,
    data_loader, 
    optimizer, 
    clip=1, 
    teacher_forcing_ratio=0.5, 
    print_every=None  # None不打印
    ):
    model.predict = False
    model.train()

    if print_every == 0:
        print_every = 1

    print_loss_total = 0  # 每次打印都重置
    start = time.time()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):

        # shape = [seq_len, batch]
        input_batchs = batch["src"]
        target_batchs = batch["trg"]
        # list
        input_lens = batch["src_len"]
        target_lens = batch["trg_len"]
        
        optimizer.zero_grad()
        
        loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio)
        print_loss_total += loss.item()
        epoch_loss += loss.item()
        loss.backward()

        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if print_every and (i+1) % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [23]:
def evaluate(
    model,
    data_loader, 
    print_every=None
    ):
    model.predict = False
    model.eval()
    if print_every == 0:
        print_every = 1

    print_loss_total = 0  # 每次打印都重置
    start = time.time()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):

            # shape = [seq_len, batch]
            input_batchs = batch["src"]
            target_batchs = batch["trg"]
            # list
            input_lens = batch["src_len"]
            target_lens = batch["trg_len"]

            loss = model(input_batchs, input_lens, target_batchs, target_lens, teacher_forcing_ratio=0)
            print_loss_total += loss.item()
            epoch_loss += loss.item()

            if print_every and (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [25]:
def summarize(
    model,
    sample, 
    idx2token=None
    ):
    model.predict = True
    model.eval()

    # shape = [seq_len, 1]
    input_batch = sample["src"]
    # list
    input_len = sample["src_len"]

    output_tokens = model(input_batch, input_len)
    output_tokens = [idx2token[t] for t in output_tokens]

    return "".join(output_tokens)

## 开始训练

In [35]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
# 超参数
BATCH_SIZE = 5
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 200
CLIP = 1

bidirectional = True
attn_method = "general"
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT, bidirectional)
dec = AttnDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional, attn_method)
model = Seq2Seq(enc, dec, device, token2idx=TEXT.vocab.stoi).to(device)
for name, param in model.named_parameters():
  print(name, '\t\t', param.shape)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

encoder.embedding.weight 		 torch.Size([245404, 256])
encoder.gru.weight_ih_l0 		 torch.Size([1536, 256])
encoder.gru.weight_hh_l0 		 torch.Size([1536, 512])
encoder.gru.bias_ih_l0 		 torch.Size([1536])
encoder.gru.bias_hh_l0 		 torch.Size([1536])
encoder.gru.weight_ih_l0_reverse 		 torch.Size([1536, 256])
encoder.gru.weight_hh_l0_reverse 		 torch.Size([1536, 512])
encoder.gru.bias_ih_l0_reverse 		 torch.Size([1536])
encoder.gru.bias_hh_l0_reverse 		 torch.Size([1536])
encoder.gru.weight_ih_l1 		 torch.Size([1536, 1024])
encoder.gru.weight_hh_l1 		 torch.Size([1536, 512])
encoder.gru.bias_ih_l1 		 torch.Size([1536])
encoder.gru.bias_hh_l1 		 torch.Size([1536])
encoder.gru.weight_ih_l1_reverse 		 torch.Size([1536, 1024])
encoder.gru.weight_hh_l1_reverse 		 torch.Size([1536, 512])
encoder.gru.bias_ih_l1_reverse 		 torch.Size([1536])
encoder.gru.bias_hh_l1_reverse 		 torch.Size([1536])
decoder.embedding.weight 		 torch.Size([245404, 256])
decoder.gru.weight_ih_l0 		 torch.Size([1536, 256]

In [36]:
# 数据集
train_set = SummarizationDataset(summary_num_list, text_num_list)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, collate_fn=padding_batch)

In [37]:
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, CLIP)
    valid_loss = evaluate(model, train_loader)
    end_time = time.time()
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "attn-model.pt")

    if epoch %2 == 0:
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

RuntimeError: ignored

In [ ]:
print("best valid loss：", best_valid_loss)
# 加载最优权重
model.load_state_dict(torch.load("en2ch-attn-model.pt"))

In [ ]:
random.seed(seed)
test_list = [22329, 22322, 6518, 24235, 16669, 16036, 17132, 13479, 15243, 19545]
for i in test_list:  # 随机看10个
    en_tokens = list(filter(lambda x: x!=0, en_num_data[i]))  # 过滤零
    ch_tokens = list(filter(lambda x: x!=3 and x!=0, ch_num_data[i]))  # 和机器翻译作对照
    sentence = [id2en[t] for t in en_tokens]
    print("【原文】")
    print("".join(sentence))
    translation = [id2ch[t] for t in ch_tokens]
    print("【原文】")
    print("".join(translation))
    test_sample = {}
    test_sample["src"] = torch.tensor(en_tokens, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(en_tokens)]
    print("【机器翻译】")
    print(translate(model, test_sample, id2ch), end="\n\n")